In [48]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [49]:
train_data = pd.read_csv('../content/train.csv')
train_data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [50]:
# Removing Special Characters
remove_special_chars = lambda text: re.sub(r'[^a-zA-Z0-9\s]', '', text)
train_data['comment_text'] = ((train_data['comment_text'].apply(remove_special_chars)).str.replace('\n', ' ')).str.lower().str.strip()

In [51]:
# Removing stopwords

import nltk
from nltk.corpus import stopwords

subset_data = train_data.head(20000)

# Download the stopwords corpus
nltk.download('punkt')
nltk.download('stopwords')

# Get the set of stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stop words from a given text
def remove_stop_words(text):
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# Apply the remove_stop_words function to the 'comment_text' column
train_data['comment_text'] = train_data['comment_text'].apply(remove_stop_words)

print(train_data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                      id                                       comment_text  \
0       0000997932d777bf  explanation edits made username hardcore metal...   
1       000103f0d9cfb60f  daww matches background colour im seemingly st...   
2       000113f07ec002fd  hey man im really trying edit war guy constant...   
3       0001b41b1c6bb37e  cant make real suggestions improvement wondere...   
4       0001d958c54c6e35                sir hero chance remember page thats   
...                  ...                                                ...   
159566  ffe987279560d7ff  second time asking view completely contradicts...   
159567  ffea4adeee384e90     ashamed horrible thing put talk page 128611993   
159568  ffee36eab5c267c9  spitzer umm theres actual article prostitution...   
159569  fff125370e4aaaf3  looks like actually put speedy first version d...   
159570  fff46fc426af1f9a  really dont think understand came idea bad rig...   

        toxic  severe_toxic  obscene  threat  insul

In [52]:
cols_target = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

subset_data = train_data.head(20000)
X_train, X_test, y_train, y_test = train_test_split(subset_data['comment_text'], subset_data[cols_target], test_size= 0.3)


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf

X = subset_data['comment_text'].values
y = subset_data[cols_target].values  # Assuming the labels start from the 3rd column

In [54]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

max_features=100000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)

In [56]:
maxpadlen = 200
X_t=pad_sequences(list_tokenized_train, maxlen=maxpadlen, padding = 'post')
X_te=pad_sequences(list_tokenized_test, maxlen=maxpadlen, padding = 'post')

Reference:

1. https://towardsdatascience.com/toxic-comment-classification-using-lstm-and-lstm-cnn-db945d6b7986

2. https://towardsdatascience.com/classifying-toxicity-in-online-comment-forums-end-to-end-project-57720af39d0b